In [1]:
%pip install matplotlib
import blocksci
import matplotlib.pyplot as plt
import matplotlib.ticker
import collections
import pandas as pd
import numpy as np
from pathlib import Path
%matplotlib inline

parser_data_directory = Path("/mnt/small-emu/small_emu.json")
cluster_directory = Path("/mnt/anal/cluster/")
dumplings_directory = Path("/mnt/dumplings/")

chain = blocksci.Blockchain(str(parser_data_directory))

from typing import Tuple

def get_block_height_for_date(date: str) -> int:
    return chain.range(date)[0].height

def get_block_height_range(start: str, end: str) -> Tuple[int, int]:
    return get_block_height_for_date(start), get_block_height_for_date(end)

Note: you may need to restart the kernel to use updated packages.
You are using configuration for testnet/regtest chain.
Setting block boundaries used to detect CoinJoins to 0...


In [2]:
print(len(chain.blocks))
print(chain.blocks.txes.to_list()[0].hash)
chain.filter_in_keys({"4a5e1e4baab89f3a32518a88c31bc87f618f76673e2cc77ab2127b7afdeda33b": 0}, 0, len(chain))

505
4a5e1e4baab89f3a32518a88c31bc87f618f76673e2cc77ab2127b7afdeda33b
Size of keys: 1


[Tx(len(txins)=0, len(txouts)=1, size_bytes=204, block_height=0, tx_index=0)]

In [3]:
%time ww2_cjs = chain.filter_coinjoin_txes(0, len(chain), "ww2")
%time ww2_cjs_hashes = set(str(tx.hash) for tx in ww2_cjs)
god_set_of_all_coinjoins = ww2_cjs_hashes
print(len(ww2_cjs))

Condition 1
Condition 2
Condition 3
CPU times: user 17 ms, sys: 91.9 ms, total: 109 ms
Wall time: 20 ms
CPU times: user 74 µs, sys: 0 ns, total: 74 µs
Wall time: 78.2 µs
38
Condition 1
Condition 2
Condition 3
Condition 1
Condition 2
Condition 3
Condition 1
Condition 2
Condition 3
Condition 1
Condition 2
Condition 1Condition 3

Condition 2
Condition 3
Condition 1
Condition 2
Condition 3
Condition 4
Condition 1
Condition 2
Condition 3
Condition 4
Condition 5
Condition 6
Condition 7
Condition 1
Condition 2
Condition 3
Condition 1
Condition 2
Condition 3
Condition 1
Condition 2
Condition 3
Condition 1
Condition 2
Condition 3
Condition 1
Condition 2
Condition 3
Condition 1
Condition 1Condition 2
Condition 3

Condition 2
Condition 3
Condition 1
Condition 2
Condition 3
Condition 1
Condition 2
Condition 3
Condition 1
Condition 2
Condition 3
Condition 1
Condition 2
Condition 3
Condition 1
Condition 2
Condition 3
Condition 4
Condition 1
Condition 2
Condition 3
Condition 4
Condition 5
Condition 6

In [4]:
import copy
import json

wp_stats = {
    "all_txes": 0,
    "all_inputs": 0,
    "all_inputs_value": 0,
    "fresh_inputs": 0,
    "fresh_inputs_value": 0,
    "all_outputs": 0,
    "remixed_outputs": 0,
    "unmoved_outputs": 0,
    "unmoved_outputs_value": 0,
    "traverse_inputs": 0,
    "traverse_inputs_value": 0
}

ww2_stats = copy.deepcopy(wp_stats)
ww2_other_stats = copy.deepcopy(wp_stats)
ww1_stats = copy.deepcopy(wp_stats)

whirlpool_stats = {
    "all": copy.deepcopy(wp_stats),
    "5m": copy.deepcopy(wp_stats),
    "1m": copy.deepcopy(wp_stats),
    "50m": copy.deepcopy(wp_stats),
    "100k": copy.deepcopy(wp_stats),
}

def get_pool(ws, tx):
    max_pool_value = max(x.value for x in tx.outputs)
    if max_pool_value == 5000000:
        return ws["5m"]
    if max_pool_value == 1000000:
        return ws["1m"]
    if max_pool_value == 50000000:
        return ws["50m"]
    if max_pool_value == 100000:
        return ws["100k"]

    raise ValueError(f"wtf {max_pool_value}")

def check_fdnp(tx: blocksci.Tx) -> int:
    s = 0
    
    for i in tx.inputs:
        if str(i.spent_tx.hash) in ww2_cjs_hashes or str(i.spent_tx.hash) in ww2_cjs_other_hashes:
            s += i.value

    return s


def should_ignore_input(tx):
    s = 0
    rv = False
    for i in tx.inputs:
        sp_tx = i.spent_tx
        #if str(sp_tx.hash) in ww1_cjs_hashes:
        #    s += i.value
        #    rv = True
        #elif str(i.spent_tx.hash) in ww2_cjs_hashes or str(i.spent_tx.hash) in ww2_cjs_other_hashes:
        #    s += i.value
        #    rv = True
    return s, rv

from collections import defaultdict
from typing import Dict, Set, List


def get_stats_for_cj(
    the_stats: Dict[str, int], 
    tx_objects: List[blocksci.Tx], 
    given_cjs: Set[str], 
    is_whirlpool: bool = False, 
    is_ww2: bool = False, 
    is_ww1: bool = False
):
    tx: blocksci.Tx
    tx_input: blocksci.Input
    tx_output: blocksci.Output
    dd_int = defaultdict(int)

    for tx in tx_objects:
        # filter out false positives
        if str(tx.hash) not in given_cjs:
            continue

    
        if is_whirlpool:
            tx_pool = get_pool(the_stats, tx)
            current_stats = the_stats["all"]
        else:
            tx_pool = dd_int
            current_stats = the_stats

        current_stats["all_txes"] += 1
        tx_pool["all_txes"] += 1
        
        current_stats["all_inputs"] += tx.input_count
        for tx_input in tx.inputs:
            current_stats["all_inputs_value"] += tx_input.value
            
            if str(tx_input.spent_tx.hash) not in god_set_of_all_coinjoins:
                how_much = 0
                ignore = False
                if is_ww1 or is_ww2:
                    how_much, ignore = should_ignore_input(tx_input.spent_tx)
  
                current_stats["fresh_inputs"] += (0 if ignore else 1) 
                tx_pool["fresh_inputs"] += 1 
                current_stats["fresh_inputs_value"] += (tx_input.value if not ignore else 0)
                tx_pool["fresh_inputs_value"] += tx_input.value

            if str(tx_input.spent_tx.hash) not in given_cjs and str(tx_input.spent_tx.hash) in god_set_of_all_coinjoins:
                current_stats["traverse_inputs"] += 1
                tx_pool["traverse_inputs"] += 1
                current_stats["traverse_inputs_value"] += tx_input.value
                tx_pool["traverse_inputs_value"] += tx_input.value

        current_stats["all_outputs"] += tx.output_count
        tx_pool["all_outputs"] += tx.output_count
        for tx_output in tx.outputs:
            if not tx_output.is_spent:
                current_stats["unmoved_outputs"] += 1
                tx_pool["unmoved_outputs"] += 1
                current_stats["unmoved_outputs_value"] += tx_output.value
                tx_pool["unmoved_outputs_value"] += tx_output.value
                continue
                
            if str(tx_output.spending_tx.hash) in god_set_of_all_coinjoins:
                current_stats["remixed_outputs"] += 1  
                tx_pool["remixed_outputs"] += 1  

In [5]:
%time get_stats_for_cj(ww2_stats, ww2_cjs, ww2_cjs_hashes, is_ww2=True)

CPU times: user 152 ms, sys: 942 µs, total: 153 ms
Wall time: 155 ms


In [6]:
from tabulate import tabulate

print(tabulate(
    [
        [
            "Wasabi 2.0 (zkSNACKs)", 
            "2022-06-18 – 2024-06-01", 
            len(ww2_cjs_hashes), 
            f"{ww2_stats['fresh_inputs'] // 1000}k/{round(ww2_stats['fresh_inputs_value'] / 100000000, 0)} BTC", 
            f"{round((ww2_stats['remixed_outputs'] / ww2_stats['all_outputs']) * 100, 1)}%", 
            f"{round((ww2_stats['unmoved_outputs'] / (ww2_stats['all_outputs'] - ww2_stats['remixed_outputs'])) * 100, 1)}%, {round(ww2_stats['unmoved_outputs_value'] / 100000000, 0)}"        
        ],
        
    ],
    headers=['Pool', 'Operating period', "Total cjtxs", "Fresh inputs", "Remix rate", "Unmoved UTXOs"], 
    tablefmt='orgtbl'
    )
)


ModuleNotFoundError: No module named 'tabulate'

In [16]:
denoms = sorted([100000000000, 94143178827, 68719476736, 62762119218, 50000000000, 34359738368, 31381059609, 20920706406, 20000000000, 
17179869184, 10460353203, 10000000000, 8589934592, 6973568802, 5000000000, 4294967296, 3486784401, 2324522934, 
2147483648, 2000000000, 1162261467, 1073741824, 1000000000, 774840978, 536870912, 500000000, 387420489, 268435456, 
258280326, 200000000, 134217728, 129140163, 100000000, 86093442, 67108864, 50000000, 43046721, 33554432, 28697814, 
20000000, 16777216, 14348907, 10000000, 9565938, 8388608, 5000000, 4782969, 4194304, 3188646, 2097152, 2000000, 
1594323, 1062882, 1048576, 1000000, 531441, 524288, 500000, 354294, 262144, 200000, 177147, 131072, 118098, 100000, 
65536, 59049, 50000, 39366, 32768, 20000, 19683, 16384, 13122, 10000, 8192, 6561, 5000])

denoms2 = sorted([
    200000000000, 20000000000, 2000000000, 20000000, 200000, 100000000000, 20000, 10000000000, 1000000000, 10000000, 
    100000, 6561, 59049, 354294, 177147, 10000, 5000, 6973568802, 134375000000, 86093442, 2324522934, 
    3188646, 3486784401, 2000000, 1073741824, 387420489, 8388608, 65536, 19683, 2147483648, 774840978, 
    16777216, 131072, 4294967296, 33554432, 262144, 8589934592, 67108864, 524288, 17179869184, 134217728, 
    8192, 1048576, 100000000, 531441, 268435456, 34359738368, 16384, 2097152, 200000000, 1062882, 1000000, 
    68719476736, 536870912, 32768, 4194304, 1162261467, 1594323, 4782969, 14348907, 43046721, 129140163, 10460353203, 
    31381059609, 94143178827, 13122, 39366, 118098, 9565938, 28697814, 258280326, 20920706406, 62762119218, 188286357654])

print(len(denoms), len(denoms2))

extra = (set(denoms) - set(denoms2)) | (set(denoms2) - set(denoms))

print(list(denoms2))

vv2 = set([5000, 6561, 8192, 10000, 13122, 16384, 19683, 20000, 32768, 39366, 50000, 59049, 65536, 100000, 118098, 131072, 177147, 200000, 262144, 354294, 500000, 524288, 531441, 1000000, 1048576, 1062882, 1594323, 2000000, 2097152, 3188646, 4194304, 4782969, 5000000, 8388608, 9565938, 10000000, 14348907, 16777216, 20000000, 28697814, 33554432, 43046721, 50000000, 67108864, 86093442, 100000000, 129140163, 134217728, 200000000, 258280326, 268435456, 387420489, 500000000, 536870912, 774840978, 1000000000, 1073741824, 1162261467, 2000000000, 2147483648, 2324522934, 3486784401, 4294967296, 5000000000, 6973568802, 8589934592, 10000000000, 10460353203, 17179869184, 20000000000, 20920706406, 31381059609, 34359738368, 50000000000, 62762119218, 68719476736, 94143178827, 100000000000, 137438953472])

print(extra - vv2)

78 74
[5000, 6561, 8192, 10000, 13122, 16384, 19683, 20000, 32768, 39366, 59049, 65536, 100000, 118098, 131072, 177147, 200000, 262144, 354294, 524288, 531441, 1000000, 1048576, 1062882, 1594323, 2000000, 2097152, 3188646, 4194304, 4782969, 8388608, 9565938, 10000000, 14348907, 16777216, 20000000, 28697814, 33554432, 43046721, 67108864, 86093442, 100000000, 129140163, 134217728, 200000000, 258280326, 268435456, 387420489, 536870912, 774840978, 1000000000, 1073741824, 1162261467, 2000000000, 2147483648, 2324522934, 3486784401, 4294967296, 6973568802, 8589934592, 10000000000, 10460353203, 17179869184, 20000000000, 20920706406, 31381059609, 34359738368, 62762119218, 68719476736, 94143178827, 100000000000, 134375000000, 188286357654, 200000000000]
{134375000000, 200000000000, 188286357654}


In [8]:
a = ['f92138403a8d1ef30e3df15910640e30d63a937a514854a21dac6ee660675534', '7c44858668109618d57f0395339c8e8600711cc948fc5921aa572c3ef758ed5e', 'b884e86783ef2e785edd4f49ce4e7ce009f3005a4f4a39009281a7778618fee7', 'ebc7462eae98e108e65dd06296d1fb1cdac4c01692978d9fee9d7661749a2343', '30b38c493aa3905e5f5629f3843d03640e4fb3f2f81ce484f5bcf0f2da443235', '6f7add9eefc8965b531841e7e2ad602b0e0c6caaa9720ff2adc91f7d37cce138', '1c6bf79c5c049a5a71d7860b530ab5815749911638def0edb204e8f843149fdf', '88c88cc9e3b0b50151eeee9c77e536433f29272f738aea8901b37628749de55c', 'c390673eab0c77305f00e1d51ee049a15a65cfb7d2ab1b3041f37ca789db434d', '55ebe4eca9d22d19dfab12c5984524c8b07966eaa64c1417f4d988caaf28c469', '3b695600c80c5df3dcd5c6706d81bf735c658f278e00ba7c9eef1973044a6d35', '1ac0a2f866157563f29bf8991bf340bd2828a1da799b764f5cea34b96a023041', '0d798b09495871bbcf844742cc9c8dc040d0b67ec5e62533de1b3dfced2223ac', '8b40b335f19acd3594d04890bf4774f275f58b9cb33fd4ba5d9e18f612f2e645', '5db57e6e62e5baeb7783508f03bf8314daad97b41c8f7968078067cfb7235e25', 'ba67a6e04e277389e271b0012486e859ebb5efcb34a740d5cdd43fada7ef5ca5', 'a1ac57993dd77308e889214fdc74c9fa45fceb8aa34660883cadf598525a237b', 'fdef351a6fbdd54e86b5c119229708348aeb36601134674f7fad825006656a96', 'b3f714aec5360cedb7173a178bdc1eba18cae19acda7771d678178a46caa8a13', '5d0d958e2793ba832cf5add4ddd88d016dbe97011e6f6a50d81722ea4bf6cae9', 'a45e403ad713f04113766492329934b84a8faf27cca6f541b06c9d0968b12fd7', 'dfcd9cab1a0ea26321fb1682485ad841d16ba981392e0ffd3ddb9548cc71f908', '6e904a94a7dd6a3e0d32a3c0c530752cc206f902535f4044946b43dbcab6206d', '02ca0c23b906656eb01c8b035e48046af9bab42273fb99a8ad2dbf5d8bdca89d', '3b854bcb7eaba5be0eb394cadad5df99a3fca7952638a5c1aadd915b90a9fa10', '2956fce983cd5fcb33dd79f3215e59200c2b115a5a8a73a6176b5b0284ebe204', '8f765eecfb7b990cb28d99a34cfc180f33fc9dc074bacd9c86497a45174c52da', 'eacbd519bfa5cae2d4511ea781199c755ca44fb5148657121c13d205075ae032', '2047555298b5d52959a64af2b8a7433d449cdf2e6bfac204c71746da385f13d9', 'e77ee3c8f7907c229209212fd745a19a79813e31f84ba090846d1e017e0c6a39', '0bebf5f71712633885d0059b4378ce6335198beb3fac380ad58fc9b068fb97e8', 'db1286ccb872c89c87a0d7465dcf92453affde602bb18eab3e54d75829e2006e', '5b67e25e132986d4b2911082902c0abd151990d43d143c7307a2a6b4975541ee', '47caef0ce19b9c1ad5d889156ea3a76b7fb5491a41487f98da79e29f402f293e', 'd9adfca8d07245fcbabe3e6d9507fee89f3dae015a1428e654d8044e6b9532dd', 'fe23d44654ef61bf168e6291d8afb2febd81005c9460c54566962f7f0fdf974f', 'a0c403aeb8880b99b094dc028dd76b60d138def60a51cdac4bd32935bb52b169', '881180bedbb76ebc8d31f9e6fb33f90f8cd0eb8a82a93dc105b66ec217ff1e04', '80bdc55bcb860b4bf24c5589e6bb6dffe9bc70af28d9696c470e3e7ca6a83547']
a = set(a)

print(a - ww2_cjs_hashes)

{'fdef351a6fbdd54e86b5c119229708348aeb36601134674f7fad825006656a96'}


In [12]:
tx = chain.tx_with_hash("fdef351a6fbdd54e86b5c119229708348aeb36601134674f7fad825006656a96")
print(type(tx.inputs[0].address_type))
print(tx.is_wasabi2_coinjoin)


inp = list(map(lambda i: i.value, tx.inputs))
out = list(map(lambda o: o.value, tx.outputs))

print(inp == sorted(inp, reverse=True))
print(out == sorted(out, reverse=True))

print(all(inp.address_type == blocksci.address_type.witness_pubkeyhash for inp in tx.inputs))
print(all(out.address_type == blocksci.address_type.witness_pubkeyhash for out in tx.outputs))

c = 0
c2 = 0
values = set()
for o in out:
    if o in denoms2:
        c += 1
    if o in extra:
        c2 += 1
    else:
        values.add(o)

print("AAA", c > tx.output_count * 0.77, c, tx.output_count, tx.output_count * 0.77, c2)

print(values)
print(all(v not in extra for v in values))

<class 'blocksci.address_type'>
False
True
True
True
True
AAA True 182 235 180.95000000000002 36
{2097152, 262144, 1062882, 200000, 131072, 100000, 3188646, 65536, 39366, 639945, 32768, 13518, 9565938, 20000, 354294, 16384, 4782969, 13887}
True
Condition 1
Condition 2
Condition 3
Condition 4
Condition 5
Condition 6
Condition 7
Condition 8, 182, 235, 188
